In [1]:
import os
import sys
os.chdir('..')
current_dir = os.path.abspath('')
sys.path.append(current_dir)

from scripts.helpers import *
from pytorch_lightning import seed_everything

c:\Users\Owner\anaconda3\envs\latent-space-probe\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed_everything(42)
SAVE_PATH = "outputs/txt2img_test/"
version_dict = {
    "H": 1024,
    "W": 1024,
    "C": 4,
    "f": 8,
    "is_legacy": False,
    "config": "configs/inference/sd_xl_base.yaml",
    "ckpt": "checkpoints/sd_xl_base_1.0.safetensors",
}

Global seed set to 42


In [3]:
config = version_dict["config"]
ckpt = version_dict["ckpt"]
config = OmegaConf.load(config)
model = load_model_from_config(config, ckpt if ckpt else None, verbose = False)

SpatialTransformer: Found context dims [2048] of depth 1, which does not match the specified 'depth' of 2. Setting context_dim to [2048, 2048] now.
SpatialTransformer: Found context dims [2048] of depth 1, which does not match the specified 'depth' of 2. Setting context_dim to [2048, 2048] now.
SpatialTransformer: Found context dims [2048] of depth 1, which does not match the specified 'depth' of 10. Setting context_dim to [2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048] now.
SpatialTransformer: Found context dims [2048] of depth 1, which does not match the specified 'depth' of 10. Setting context_dim to [2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048] now.
SpatialTransformer: Found context dims [2048] of depth 1, which does not match the specified 'depth' of 10. Setting context_dim to [2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048] now.
SpatialTransformer: Found context dims [2048] of depth 1, which does not match the specified 'depth' of 10. Setti

Initialized embedder #0: FrozenCLIPEmbedder with 123060480 params. Trainable: False
Initialized embedder #1: FrozenOpenCLIPEmbedder2 with 694659841 params. Trainable: False
Initialized embedder #2: ConcatTimestepEmbedderND with 0 params. Trainable: False
Initialized embedder #3: ConcatTimestepEmbedderND with 0 params. Trainable: False
Initialized embedder #4: ConcatTimestepEmbedderND with 0 params. Trainable: False
Loading model from checkpoints/sd_xl_base_1.0.safetensors


In [4]:
prompt = ""
negative_prompt = "God"
sampler, num_rows, num_cols = init_sampling()
num_samples = num_rows * num_cols

In [5]:
init_dict = {
    "orig_width": version_dict["W"],
    "orig_height": version_dict["H"],
    "target_width": version_dict["W"],
    "target_height": version_dict["H"],
}
value_dict = init_embedder_options(
    get_unique_embedder_keys_from_conditioner(model.conditioner),
    init_dict,
    prompt=prompt,
    negative_prompt=negative_prompt,
)

In [6]:
out = do_sample(
    model,
    sampler,
    value_dict,
    num_samples,
    version_dict["H"],
    version_dict["W"],
    version_dict["C"],
    version_dict["f"],
    force_uc_zero_embeddings=["txt"] if not version_dict["is_legacy"] else [],
    return_latents=False,
    filter=None,
)

##############################  Sampling setting  ##############################
Sampler: EulerEDMSampler
Discretization: LegacyDDPMDiscretization
Guider: VanillaCFG


Sampling with EulerEDMSampler for 51 steps:   0%|          | 0/51 [00:00<?, ?it/s]c:\Users\Owner\anaconda3\envs\latent-space-probe\lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
Sampling with EulerEDMSampler for 51 steps:  98%|█████████▊| 50/51 [00:08<00:00,  6.05it/s]


In [7]:
perform_save_locally(SAVE_PATH, out)